# BERT on Repo Description

1. Construct a sentence corpus for each software type using labeled/manually validated repo descriptions
2. Calculate embeddingfor each corpus
3. Compare all repo description with each software type corpus using cosine-similarity score
    - It took about 1.5 hrs to run the embedding on repo data

Author: Cierra and Crystal

In [12]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [19]:
#pgadmin
import os
import psycopg2 as pg


#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd
import numpy as np

import re

import nltk
nltk.download("punkt")

from nltk import tokenize

import scipy

import datetime

[nltk_data] Downloading package punkt to /home/zz3hs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Embedding Model

In [20]:
#embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2') #quicker model
embedder = SentenceTransformer('paraphrase-mpnet-base-v2') #most accurate, long run time

# Data

## I. Unlabelled Repo Data

In [4]:
repo_data = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/clean_github_repos_157k.csv")
len(repo_data)

repo_data = repo_data[repo_data.description.notnull()]
len(repo_data)

155811

In [21]:
np_repo_embedding = np.load('/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_description_embeddings.npy')

In [22]:
len(np_repo_embedding[0])

768

## II. Labelled Repo Data -- Software Type Corpus

In [7]:
data = pd.read_excel('/home/zz3hs/git/dspg21oss/data/dspg21oss/oss_software_labelled_blockchain_cz.xlsx') #import csv
#data = pd.read_excel('/home/zz3hs/git/dspg21oss/data/dspg21oss/oss_software_labelled_dataviz_cz.xlsx') #import csv

data

<ipython-input-7-857f48263746>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  data = pd.read_excel('/home/zz3hs/git/dspg21oss/data/dspg21oss/oss_software_labelled_blockchain_cz.xlsx') #import csv


,Unnamed: 0,slug,blockchain_label,app_blockchain_all,description,readme,language,topics,commits,forks,stars,watchers,description_clean1,description_cleaneng,blockchain_label2
0,1,bitcoin/bitcoin,1.0,1,Bitcoin Core integration/staging tree,Bitcoin is an experimental digital currency th...,C++,"['bitcoin', 'c-plus-plus', 'p2p', 'cryptocurre...",22581,29381,55596,3827,Bitcoin Core integration staging tree,Bitcoin Core integration staging tree,1.0
1,2,ccxt/ccxt,1.0,2,A JavaScript / Python / PHP cryptocurrency tra...,# CCXT – CryptoCurrency eXchange Trading Libra...,JavaScript,"['altcoin', 'api', 'arbitrage', 'bitcoin', 'bo...",27607,4971,20017,951,A JavaScript Python PHP cryptocurrency tra...,A JavaScript Python PHP cryptocurrency tra...,0.0
2,3,DeviaVir/zenbot,1.0,1,Zenbot is a command-line cryptocurrency tradin...,a,HTML,"['zenbot', 'trade', 'paper-trading', 'strategy...",2778,2001,7777,491,Zenbot is a command line cryptocurrency tradin...,Zenbot is a command line cryptocurrency tradin...,1.0
3,4,butor/blackbird,1.0,1,Blackbird Bitcoin Arbitrage: a long/short mark...,Blackbird Bitcoin Arbitrage is a C++ trading s...,C++,"['bitcoin', 'arbitrage', 'market-neutrality', ...",513,1015,5330,404,Blackbird Bitcoin Arbitrage a long short mark...,Blackbird Bitcoin Arbitrage a long short mark...,1.0
4,5,spesmilo/electrum,1.0,2,Electrum Bitcoin Wallet,Electrum - Lightweight Bitcoin client\n=======...,Python,"['bitcoin', 'bitcoin-wallet', 'python', 'elect...",11836,2488,5017,311,Electrum Bitcoin Wallet,Electrum Bitcoin Wallet,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,zone117x/node-open-mining-portal,NaN,1,A scalable all-in-one easy to setup cryptocurr...,## This repo is looking for maintainers! Pleas...,JavaScript,[],547,963,902,163,A scalable all in one easy to setup cryptocurr...,A scalable all in one easy to setup cryptocurr...,NaN
65,66,Bit-Wasp/bitcoin-php,1.0,1,Bitcoin implementation in PHP,\n## Bitcoin\n\n[![Build Status](https://scrut...,PHP,"['php', 'bitcoin', 'electrum', 'bip32', 'trans...",2806,296,895,76,Bitcoin implementation in PHP,Bitcoin implementation in PHP,NaN
66,67,lightninglabs/lightning-app,1.0,1,An easy-to-use cross-platform Lightning wallet,Lightning [![Build Status](https://travis-ci.o...,JavaScript,"['lightning', 'cryptocurrency', 'bitcoin', 'el...",3221,184,880,74,An easy to use cross platform Lightning wallet,An easy to use cross platform Lightning wallet,NaN
67,68,trustwallet/wallet-core,NaN,2,"Cross-platform, cross-blockchain wallet library.","<img align=""center"" src=""docs/banner.png"" titl...",C++,"['blockchain', 'cross-platform', 'bitcoin', 'e...",529,661,877,66,Cross platform cross blockchain wallet library,Cross platform cross blockchain wallet library,NaN


In [50]:
# software type
type_name =  "blockchain_label2"
#type_name =  "dataviz_label2"
#filter 500 validated repos that are labelled 1 (numeric)
corpus_type_i = data[data[type_name] ==1][["slug",type_name, "description_cleaneng", "readme"]]

In [51]:
corpus_type_i

,slug,blockchain_label2,description_cleaneng,readme
0,bitcoin/bitcoin,1.0,Bitcoin Core integration staging tree,Bitcoin is an experimental digital currency th...
2,DeviaVir/zenbot,1.0,Zenbot is a command line cryptocurrency tradin...,a
3,butor/blackbird,1.0,Blackbird Bitcoin Arbitrage a long short mark...,Blackbird Bitcoin Arbitrage is a C++ trading s...
5,btcsuite/btcd,1.0,An alternative full node bitcoin implementatio...,"It properly downloads, validates, and serves t..."
8,bitpay/bitcore,1.0,A full stack for bitcoin and blockchain based ...,Infrastructure to build Bitcoin and blockchain...
9,bitcoinj/bitcoinj,1.0,A library for working with Bitcoin,The bitcoinj library is a Java implementation ...
10,paritytech/polkadot,1.0,Polkadot Node Implementation,Polkadot enables scalability by allowing speci...
11,corda/corda,1.0,Corda is an open source blockchain project de...,a
13,bisq-network/bisq,1.0,A decentralized bitcoin exchange network,"Bisq is a safe, private and decentralized way ..."
16,nanocurrency/nano-node,1.0,Nano is a cryptocurrency,Nano is a digital payment protocol designed to...


In [52]:
# description sentence token
desc_tokens = corpus_type_i["description_cleaneng"].tolist()
desc_tokens

['Bitcoin Core integration staging tree',
 'Zenbot is a command line cryptocurrency trading bot using Node js and MongoDB',
 'Blackbird Bitcoin Arbitrage  a long short market neutral strategy',
 'An alternative full node bitcoin implementation written in Go  golang',
 'A full stack for bitcoin and blockchain based applications',
 'A library for working with Bitcoin',
 'Polkadot Node Implementation',
 'Corda is an open source blockchain project  designed for business from the start  Only Corda allows you to build interoperable blockchain networks that transact in strict privacy  Corda s smart contract technology allows businesses to transact directly  with value',
 'A decentralized bitcoin exchange network',
 'Nano is a cryptocurrency',
 'Bitpay Wallet  formerly Copay  is a secure Bitcoin and other crypto currencies wallet platform for both desktop and mobile devices',
 'Complete Ethereum library and wallet implementation in JavaScript',
 'The Stacks 2 0 blockchain implementation',
 'A 

In [53]:
# readme sentence token

readmes =  corpus_type_i["readme"].tolist()

#tokenize.sent_tokenize(readme)
readmes_ls = []
for readme in readmes:
    readmes_ls.append(readme)
readmes_ls_filtered = [x for x in readmes_ls if x != "a"]

readmes_str = ''.join(readmes_ls_filtered) 
readmes_str

readme_tokens =tokenize.sent_tokenize(readmes_str)
readme_tokens

['Bitcoin is an experimental digital currency that enables instant payments to\nanyone, anywhere in the world.',
 'Bitcoin uses peer-to-peer technology to operate\nwith no central authority: managing transactions and issuing money are carried\nout collectively by the network.',
 'Bitcoin Core is the name of open source\nsoftware which enables the use of this currency.Blackbird Bitcoin Arbitrage is a C++ trading system that does automatic long/short arbitrage between Bitcoin exchanges.Bitcoin is still a new and inefficient market.',
 'Several Bitcoin exchanges exist around the world and the bid/ask prices they propose can be briefly different from an exchange to another.',
 'The purpose of Blackbird is to automatically profit from these temporary price differences while being market-neutral.It properly downloads, validates, and serves the block chain using the exact\nrules (including consensus bugs) for block acceptance as Bitcoin Core.',
 'We have\ntaken great care to avoid btcd causin

In [54]:
corpus_type_i_token = desc_tokens + readme_tokens
corpus_type_i_token

['Bitcoin Core integration staging tree',
 'Zenbot is a command line cryptocurrency trading bot using Node js and MongoDB',
 'Blackbird Bitcoin Arbitrage  a long short market neutral strategy',
 'An alternative full node bitcoin implementation written in Go  golang',
 'A full stack for bitcoin and blockchain based applications',
 'A library for working with Bitcoin',
 'Polkadot Node Implementation',
 'Corda is an open source blockchain project  designed for business from the start  Only Corda allows you to build interoperable blockchain networks that transact in strict privacy  Corda s smart contract technology allows businesses to transact directly  with value',
 'A decentralized bitcoin exchange network',
 'Nano is a cryptocurrency',
 'Bitpay Wallet  formerly Copay  is a secure Bitcoin and other crypto currencies wallet platform for both desktop and mobile devices',
 'Complete Ethereum library and wallet implementation in JavaScript',
 'The Stacks 2 0 blockchain implementation',
 'A 

# Embedding of corpus

In [55]:
# embedding for the corpus
corpus_type_i_embeddings = embedder.encode(corpus_type_i_token, show_progress_bar=True) # embeddings


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

# Similarity Score

takes about 35 seconds to run

In [56]:
queries = np_repo_embedding

# pre-specified number of sentences
num_sentences = 10 #find 10 most similar sentences from the corpus

# init a result list for scores
result = []


t1 = datetime.datetime.now()
print("Start:", t1)

for query in queries: #compare each sentence in the abstract to the software type corpus
    
    # We use cosine-similarity and torch.topk to find the highest k scores
    cos_scores = util.pytorch_cos_sim(query, corpus_type_i_embeddings)[0]

    top_results = torch.topk(cos_scores, k=num_sentences)   #get the top k scores
    result.append(top_results.values.tolist()) #unlist the top result list
   
    #print 10 most similar entences from the corpus and their corresponding scores
    #print("\n\n======================\n\n")
    #print("Query:", query)
    #print("Results:", top_results)
    #print("\nTop k=10 most similar sentences in corpus:")
    #for score, idx in zip(top_results[0], top_results[1]):
    #    print(corpus_type_i_clean[idx], "(Score: {:.4f})".format(score))

t2 =  datetime.datetime.now()
print("Finished", len(result), "descriptions at", t2)
print("It took", t2-t1, "to run.")

Start: 2021-08-01 19:07:43.314534
Finished 155811 descriptions at 2021-08-01 19:08:12.046770
It took 0:00:28.732236 to run.


In [57]:
#TODO: THIS WAY, similarity_score IS SAVED AS A STRING, NEED TO FIGURE OUT HOW TO SAVE AS A LIST
#save the similarity score as a variable of the original repo data
repo_data["similarity_score"] = result

In [58]:
repo_data

,slug,description,similarity_score
0,vuejs/vue,Vue js is a progressive incrementally adopt...,"[0.3805304765701294, 0.33000627160072327, 0.31..."
1,facebook/react,A declarative efficient and flexible JavaScr...,"[0.4392547011375427, 0.3838126063346863, 0.372..."
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"[0.37712961435317993, 0.35659509897232056, 0.3..."
3,twbs/bootstrap,The most popular HTML CSS and JavaScript fra...,"[0.2848309278488159, 0.2714821994304657, 0.245..."
4,ohmyzsh/ohmyzsh,A delightful community driven with 1700 c...,"[0.4313505291938782, 0.418905109167099, 0.3998..."
...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"[0.3291291296482086, 0.30754026770591736, 0.23..."
157534,dajinchu/kde-connect-android,For Google Code In,"[0.27807173132896423, 0.23858405649662018, 0.2..."
157535,LibrinnoTeam/LibraryHelpBot,Library Management System ITP2 project,"[0.45676445960998535, 0.4496637284755707, 0.42..."
157536,Twissi/Animator,Animator for hacklace See hacklace org for fu...,"[0.2707914113998413, 0.2257576286792755, 0.210..."


In [59]:
import statistics #calculate mean and others

sentence_score_ls = repo_data["similarity_score"]

mean_sentence_score= []
max_sentence_score = []
median_sentence_score = []
for sentence_score in sentence_score_ls:
    mean_sentence_score.append(statistics.mean(sentence_score))
    max_sentence_score.append(max(sentence_score))
    median_sentence_score.append(statistics.median(sentence_score))

repo_data["mean_sentence_score"]=mean_sentence_score
repo_data["max_sentence_score"]=max_sentence_score
repo_data["median_sentence_score"]=median_sentence_score


In [60]:
repo_data

,slug,description,similarity_score,mean_sentence_score,max_sentence_score,median_sentence_score
0,vuejs/vue,Vue js is a progressive incrementally adopt...,"[0.3805304765701294, 0.33000627160072327, 0.31...",0.298922,0.380530,0.286049
1,facebook/react,A declarative efficient and flexible JavaScr...,"[0.4392547011375427, 0.3838126063346863, 0.372...",0.323058,0.439255,0.297213
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"[0.37712961435317993, 0.35659509897232056, 0.3...",0.336658,0.377130,0.327444
3,twbs/bootstrap,The most popular HTML CSS and JavaScript fra...,"[0.2848309278488159, 0.2714821994304657, 0.245...",0.226962,0.284831,0.229019
4,ohmyzsh/ohmyzsh,A delightful community driven with 1700 c...,"[0.4313505291938782, 0.418905109167099, 0.3998...",0.370409,0.431351,0.353861
...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"[0.3291291296482086, 0.30754026770591736, 0.23...",0.216731,0.329129,0.204391
157534,dajinchu/kde-connect-android,For Google Code In,"[0.27807173132896423, 0.23858405649662018, 0.2...",0.201072,0.278072,0.188174
157535,LibrinnoTeam/LibraryHelpBot,Library Management System ITP2 project,"[0.45676445960998535, 0.4496637284755707, 0.42...",0.380199,0.456764,0.374620
157536,Twissi/Animator,Animator for hacklace See hacklace org for fu...,"[0.2707914113998413, 0.2257576286792755, 0.210...",0.199354,0.270791,0.199800


In [61]:
#save csv
#repo_data.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_data_blockchain_score.csv', index = False)   
#repo_data.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_data_dataviz_score.csv', index = False)   


# Similarity Score Analysis

In [18]:
from scipy import stats
from scipy.stats import skew
import statistics #calculate mean and others

In [16]:
#read in data
repo_data = pd.read_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_data_python_score.csv')   


In [17]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers,similarity_score
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adopt...","<p align=""center""><a href=""https://vuejs.org"" ...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070.0,29611.0,185611.0,6250.0,"[0.42707720398902893, 0.41736093163490295, 0.3..."
1,facebook/react,"A declarative, efficient, and flexible JavaScr...",# [React](https://reactjs.org/) · [![GitHub li...,JavaScript,"['javascript', 'react', 'frontend', 'declarati...",12695.0,34352.0,171327.0,6718.0,"[0.5678911805152893, 0.5504274368286133, 0.549..."
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"<div align=""center"">\n<img src=""https://www.te...",C++,"['tensorflow', 'machine-learning', 'python', '...",75671.0,84937.0,156754.0,8092.0,"[0.6914295554161072, 0.6374238133430481, 0.602..."
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript fra...","<p align=""center"">\n<a href=""https://getbootst...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'sc...",19228.0,73981.0,151778.0,7079.0,"[0.4793936014175415, 0.46719563007354736, 0.46..."
4,ohmyzsh/ohmyzsh,🙃 A delightful community-driven (with 1700+ c...,"<p align=""center""><img alt=""Oh My Zsh"" src=""ht...",Shell,"['shell', 'zsh-configuration', 'theme', 'termi...",5447.0,22232.0,129314.0,2678.0,"[0.4871608316898346, 0.44735753536224365, 0.44..."
...,...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"# CA Alerts, made with Very Little Gravitas fo...",CSS,"['prototype', 'messaging', 'emergency', 'govte...",414.0,1.0,0.0,2.0,"[0.34419283270835876, 0.3314574360847473, 0.32..."
157534,dajinchu/kde-connect-android,For Google Code-In,NaN,Java,[],414.0,0.0,0.0,1.0,"[0.525587260723114, 0.47061866521835327, 0.459..."
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the ap...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415.0,2.0,0.0,2.0,"[0.5185543894767761, 0.517078697681427, 0.4827..."
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for fu...,Animator\n========\n\nAnimator for hacklace. S...,Java,[],415.0,0.0,0.0,0.0,"[0.41843315958976746, 0.4162086844444275, 0.40..."


In [29]:
#score is in a string, convert to a list, also make sure the numbers are float
score_ls = repo_data["similarity_score"]

score_ls_float = []
for sentence_score in score_ls:
    sentence_score = str(sentence_score)[1:-1]
    sentence_score = sentence_score.split(",")
    item_float= []
    for item in sentence_score:
        item_float.append(float(item))
    score_ls_float.append(item_float)

    
repo_data["similarity_score_float"] = score_ls_float

In [30]:
#check scores are in a list
repo_data["similarity_score_float"][0][0]

0.42707720398902893

In [31]:
# get score statistics
score_ls = repo_data["similarity_score_float"]

mean_score= []
range_score = []
max_score = []
median_score = []
skewness_score = []
for sentence_score in score_ls:
    mean_score.append(statistics.mean(sentence_score))
    range_score.append(max(sentence_score)- min(sentence_score))
    max_score.append(max(sentence_score))
    median_score.append(statistics.median(sentence_score))
    skewness_score.append(stats.skew(sentence_score))
repo_data["mean_score"]=mean_score
repo_data["range_score"]=range_score
repo_data["max_score"]=max_score
repo_data["median_score"]=median_score
repo_data["skewness_score"]=skewness_score

In [32]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers,similarity_score,similarity_score_float,mean_score,range_score,max_score,median_score,skewness_score
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adopt...","<p align=""center""><a href=""https://vuejs.org"" ...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070.0,29611.0,185611.0,6250.0,"[0.42707720398902893, 0.41736093163490295, 0.3...","[0.42707720398902893, 0.41736093163490295, 0.3...",0.386763,0.065932,0.427077,0.381985,0.976499
1,facebook/react,"A declarative, efficient, and flexible JavaScr...",# [React](https://reactjs.org/) · [![GitHub li...,JavaScript,"['javascript', 'react', 'frontend', 'declarati...",12695.0,34352.0,171327.0,6718.0,"[0.5678911805152893, 0.5504274368286133, 0.549...","[0.5678911805152893, 0.5504274368286133, 0.549...",0.516635,0.074020,0.567891,0.503811,0.849749
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"<div align=""center"">\n<img src=""https://www.te...",C++,"['tensorflow', 'machine-learning', 'python', '...",75671.0,84937.0,156754.0,8092.0,"[0.6914295554161072, 0.6374238133430481, 0.602...","[0.6914295554161072, 0.6374238133430481, 0.602...",0.603414,0.110041,0.691430,0.587026,1.810697
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript fra...","<p align=""center"">\n<a href=""https://getbootst...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'sc...",19228.0,73981.0,151778.0,7079.0,"[0.4793936014175415, 0.46719563007354736, 0.46...","[0.4793936014175415, 0.46719563007354736, 0.46...",0.424735,0.096729,0.479394,0.413202,0.336363
4,ohmyzsh/ohmyzsh,🙃 A delightful community-driven (with 1700+ c...,"<p align=""center""><img alt=""Oh My Zsh"" src=""ht...",Shell,"['shell', 'zsh-configuration', 'theme', 'termi...",5447.0,22232.0,129314.0,2678.0,"[0.4871608316898346, 0.44735753536224365, 0.44...","[0.4871608316898346, 0.44735753536224365, 0.44...",0.432683,0.075190,0.487161,0.425684,1.349496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"# CA Alerts, made with Very Little Gravitas fo...",CSS,"['prototype', 'messaging', 'emergency', 'govte...",414.0,1.0,0.0,2.0,"[0.34419283270835876, 0.3314574360847473, 0.32...","[0.34419283270835876, 0.3314574360847473, 0.32...",0.317462,0.045936,0.344193,0.318741,0.451481
157534,dajinchu/kde-connect-android,For Google Code-In,NaN,Java,[],414.0,0.0,0.0,1.0,"[0.525587260723114, 0.47061866521835327, 0.459...","[0.525587260723114, 0.47061866521835327, 0.459...",0.366771,0.236830,0.525587,0.312547,0.700486
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the ap...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415.0,2.0,0.0,2.0,"[0.5185543894767761, 0.517078697681427, 0.4827...","[0.5185543894767761, 0.517078697681427, 0.4827...",0.471379,0.077404,0.518554,0.467471,0.701968
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for fu...,Animator\n========\n\nAnimator for hacklace. S...,Java,[],415.0,0.0,0.0,0.0,"[0.41843315958976746, 0.4162086844444275, 0.40...","[0.41843315958976746, 0.4162086844444275, 0.40...",0.400396,0.032988,0.418433,0.397456,0.503637
